In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
from agent import Agent
from dataset import Dataset
from network import QNetwork
from memory import Memory

In [3]:
class CassavaLeafDataset:
    def __init__(self, image_size = (512, 512), batch_size=32):
        self.image_size = image_size
        self.batch_size = batch_size
        self.reading_csv("../Deep-Reinforcement-Learning-on-Imbalanced-Data/cassava-leaf-disease-classification/train_images/", "../Deep-Reinforcement-Learning-on-Imbalanced-Data/cassava-leaf-disease-classification/train.csv")
        self.create_dataset()

        self.get_rho()
        self.get_minority_classes()

    def reading_csv(self, folder_path, file_path):
        df = pd.read_csv(file_path) # Load train image file names and each label data
        df["filepath"] = folder_path + df["image_id"] # Create path by adding folder name and image name for load images easily
        df = df.drop(['image_id'],axis=1) # Drop image names which is useless.
        self.X = df.drop(columns=["label"])
        self.y = df['label']
#         self.y = tf.keras.utils.to_categorical(df["label"].values)

    
    def imbalance_the_data(self):
        pass

    def load_image_and_label_from_path(self, image_path, label):
        img = tf.io.read_file(image_path)
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, self.image_size)
        return img, label
    
    def create_dataset(self):
        X_train, X_test, y_train, y_test = train_test_split(self.X, self.y, random_state=42, test_size=0.2)
        
        training_data = tf.data.Dataset.from_tensor_slices((X_train.filepath.values, y_train))
        testing_data = tf.data.Dataset.from_tensor_slices((X_test.filepath.values, y_test))

        AUTOTUNE = tf.data.experimental.AUTOTUNE

        training_data = training_data.map(self.load_image_and_label_from_path, num_parallel_calls=AUTOTUNE)
        testing_data = testing_data.map(self.load_image_and_label_from_path, num_parallel_calls=AUTOTUNE)

        self.training_data_batches = training_data.shuffle(buffer_size=1000).batch(self.batch_size).prefetch(buffer_size=AUTOTUNE)
        self.testing_data_batches = testing_data.shuffle(buffer_size=1000).batch(self.batch_size).prefetch(buffer_size=AUTOTUNE)
        
        self.x_train = []
        self.y_train = []
        self.x_test = []
        self.y_test = []
        
        # Create a TensorFlow session
        with tf.compat.v1.Session() as sess:
            train_iterator = tf.compat.v1.data.make_one_shot_iterator(self.training_data_batches)
            train_next_element = train_iterator.get_next()
        
            while True:
                try:
                    features, labels = sess.run(train_next_element)
                    for i in range(len(labels)):
                        self.x_train.append(features[i])
                        self.y_train.append((labels[i],))    
                except tf.errors.OutOfRangeError:
                    break
            
            test_iterator = tf.compat.v1.data.make_one_shot_iterator(self.testing_data_batches)
            test_next_element = test_iterator.get_next()
            
            while True:
                try:
                    features, labels = sess.run(test_next_element)
                    for i in range(len(labels)):
                        self.x_test.append(features[i])
                        self.y_test.append((labels[i],))
                        
                except tf.errors.OutOfRangeError:
                    break
        print(len(self.y_train))
        print(len(self.y_test))
        return self.training_data_batches, self.testing_data_batches

    def get_class_num(self):
        # get number of all classes
#         _, nums_cls = np.unique(np.argmax(self.y, axis=1), return_counts=True)
        _, nums_cls = np.unique(self.y, return_counts=True)
        print("No of total samples in dataset and their distribution: ", np.unique(self.y, return_counts=True))
        
        return nums_cls

    def get_minority_classes(self):
        label, label_count = np.unique(self.y, return_counts=True)
        print("Label is ", label)
        labels_with_counts = {}
        for i in range(len(label)):
            labels_with_counts[label[i]] = label_count[i]
        print("Labels with count",labels_with_counts)
        labels_with_counts = sorted(labels_with_counts.items())

        # We are going to get 35% minority classes from total classes i-e if there are total 6 classes then we will only set 2 classes as minority classes
        no_of_minority_classes_to_get = int(np.round(len(label) * 0.35))

        self.minority_classes = []
        for i in range(no_of_minority_classes_to_get):
            self.minority_classes.append(labels_with_counts[i][0])

    def get_rho(self):
        """
        In the two-class dataset problem, research paper has proven that the best performance is achieved when the reciprocal of the ratio of the number of data is used as the reward function.
        In this code, the result of this paper is extended to multi-class by creating a reward function with the reciprocal of the number of data for each class.
        """
        nums_cls = self.get_class_num()
        raw_reward_set = 1 / nums_cls
        self.reward_set = np.round(raw_reward_set / np.linalg.norm(raw_reward_set), 6)
        print("\nReward for each class.")
        for cl_idx, cl_reward in enumerate(self.reward_set):
            print("\t- Class {} : {:.6f}".format(cl_idx, cl_reward))


class PersonalityDataset:
    def __init__(self, batch_size=100):

        self.batch_size = batch_size
        self.create_dataset()
        self.get_rho()
        self.get_minority_classes()

    def create_dataset(self):
        df = pd.read_csv("../16P/16P.csv", encoding='cp1252')
        
        df = df.dropna()

        self.X = df.drop(["Personality", "Response Id"], axis = 1)
        self.y = df["Personality"]

        self.label_encoder = LabelEncoder()
        self.y = self.label_encoder.fit_transform(self.y)
        
        self.y = tf.keras.utils.to_categorical(self.y)

        self.unique_labels, self.label_counts = np.unique((np.argmax(self.y, axis=1)), return_counts=True)
        
        X_train, X_test, y_train, y_test = train_test_split(self.X.values, self.y, random_state=42, test_size=0.2)
        
        # We are going to get 25% minority classes from total classes i-e if there are total 6 classes then we will only set 2 classes as minority classes
        self.no_of_minority_classes_to_get = int(np.round(len(np.unique(np.argmax(y_train, axis=1))) * 0.25))
        
        # Specify the percentage of label 2 data to remove
        percentage_to_remove = 90
        for class_to_remove in range(self.no_of_minority_classes_to_get):
            argmax_values = np.argmax(y_train, axis=1)
            indices_to_remove = np.unique(np.where(argmax_values == class_to_remove)[0])
            # Calculate the number of samples to remove
            num_samples_to_remove = int(percentage_to_remove / 100 * len(indices_to_remove))

            # Randomly select indices to remove
            indices_to_remove = np.random.choice(indices_to_remove, num_samples_to_remove, replace=False)
            # Remove the selected samples
            percentage_to_remove -= 10

            # Remove the selected samples
            X_train = np.delete(X_train, indices_to_remove, axis=0)
            y_train = np.delete(y_train, indices_to_remove, axis=0)
            percentage_to_remove -= 10

        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test

        self.length_of_dataset = len(X_train)
        
    def get_labels_counts(self):
        self.unique_labels, self.label_counts = np.unique(np.argmax(self.y_train, axis=1), return_counts=True)
        
        return self.label_counts
        
    def get_minority_classes(self):
        
        unique_labels, label_counts = np.unique((np.argmax(self.y_train, axis=1)), return_counts=True)
        print("Label is ", unique_labels)
        labels_with_counts = {}


        for i in range(len(unique_labels)):
            labels_with_counts[unique_labels[i]] = label_counts[i]
        
        print("Labels with count",labels_with_counts)
        labels_with_counts = sorted(labels_with_counts.items())

        # We are going to get 35% minority classes from total classes i-e if there are total 6 classes then we will only set 2 classes as minority classes
        no_of_minority_classes_to_get = int(np.round(len(unique_labels) * 0.35))

        self.minority_classes = []
        for i in range(no_of_minority_classes_to_get):
            self.minority_classes.append(labels_with_counts[i][0])

    def get_rho(self):
        """
        In the two-class dataset problem, research paper has proven that the best performance is achieved when the reciprocal of the ratio of the number of data is used as the reward function.
        In this code, the result of this paper is extended to multi-class by creating a reward function with the reciprocal of the number of data for each class.
        """
        nums_cls = self.get_labels_counts()
        raw_reward_set = 1 / nums_cls
        self.reward_set = np.round(raw_reward_set / np.linalg.norm(raw_reward_set), 6)
        print("\nReward for each class.")
        for cl_idx, cl_reward in enumerate(self.reward_set):
            print("\t- Class {} : {:.6f}".format(cl_idx, cl_reward))

In [4]:
class Config:
    def __init__(self, new_class, minor_classes, train_step, restore_model_path, gamma, learning_rate, batch, 
                epsilon_range, epsilon_polynomial_decay_step, target_soft_update, target_update_step, save_term, evaluation_term,
                show_phase):
        self.new_class = new_class
        self.minor_classes = minor_classes
        self.train_step = train_step
        self.restore_model_path = restore_model_path
        self.gamma = gamma
        self.learning_rate = learning_rate
        self.batch = batch
        self.epsilon_range = epsilon_range
        self.epsilon_polynomial_decay_step = epsilon_polynomial_decay_step
        self.target_soft_update = target_soft_update
        self.target_update_step = target_update_step
        self.save_folder = '.model'
        self.save_term = save_term
        self.evaluation_term = evaluation_term
        self.show_phase = show_phase

config = Config({0:[0], 1:[1], 2:[2], 3: [3], 4:[4]}, [0, 1], 120000, '', 0.1, 0.001, 32, [0.01, 1], 120000, 1., 1000, 120000, 1000, 'Validation')

In [5]:
# print(config.new_class)

In [6]:
# dataset.x_train[0].shape

In [7]:
dataset = CassavaLeafDataset(image_size = (128, 128), batch_size=32)


17117
4280
17117
4280
No of total samples in dataset and their distribution:  (array([0, 1, 2, 3, 4], dtype=int64), array([ 1087,  2189,  2386, 13158,  2577], dtype=int64))

Reward for each class.
	- Class 0 : 0.781136
	- Class 1 : 0.387891
	- Class 2 : 0.355865
	- Class 3 : 0.064531
	- Class 4 : 0.329489
Label is  [0 1 2 3 4]
Labels with count {0: 1087, 1: 2189, 2: 2386, 3: 13158, 4: 2577}


In [8]:
q_network = QNetwork(config, 128, 'complex')
memory = Memory()
agent = Agent(q_network, dataset, memory, config)
agent.train()

C:\Users\hassa\OneDrive\Desktop\AI Degree\Knowledge Representation and Reasonins\DRL-on-Imbalanced-Data\network.py:63: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2D` instead.
  x = tf.compat.v1.layers.conv2d(self.state, 64, 5, strides=2, activation=tf.nn.relu)
C:\Users\hassa\OneDrive\Desktop\AI Degree\Knowledge Representation and Reasonins\DRL-on-Imbalanced-Data\network.py:64: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2D` instead.
  x = tf.compat.v1.layers.conv2d(x, 32, 5, strides=2, activation=tf.nn.relu)
C:\Users\hassa\OneDrive\Desktop\AI Degree\Knowledge Representation and Reasonins\DRL-on-Imbalanced-Data\network.py:65: UserWarning: `tf.layers.max_pooling2d` is deprecated and will be removed in a future version. Please use `tf.keras.layers.MaxPooling2D` instead.
  x = tf.compat.v1.layers.max_pooling2d(x, (2,2), strides=2)
C:\Users\h

start training
train_step : 1000, epsilon : 0.992
Accuracy Score:  0.10163551401869159
		 Validation Data.     f1-score of class 0 : 0.000, class 1 : 0.185, class 2 : 0.000, class 3 : 0.000, class 4 : 0.000, weighted macro avg : 0.019
train_step : 2000, epsilon : 0.984
Accuracy Score:  0.10163551401869159
		 Validation Data.     f1-score of class 0 : 0.000, class 1 : 0.185, class 2 : 0.000, class 3 : 0.000, class 4 : 0.000, weighted macro avg : 0.019
train_step : 3000, epsilon : 0.975
Accuracy Score:  0.624766355140187
		 Validation Data.     f1-score of class 0 : 0.000, class 1 : 0.000, class 2 : 0.000, class 3 : 0.769, class 4 : 0.000, weighted macro avg : 0.480
train_step : 4000, epsilon : 0.967
Accuracy Score:  0.624766355140187
		 Validation Data.     f1-score of class 0 : 0.000, class 1 : 0.000, class 2 : 0.000, class 3 : 0.769, class 4 : 0.000, weighted macro avg : 0.480
train_step : 5000, epsilon : 0.959
Accuracy Score:  0.11612149532710281
		 Validation Data.     f1-score of c

KeyboardInterrupt: 